<a href="https://colab.research.google.com/github/jrleonett/SwinIR/blob/main/SwinIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mejora de Imágenes con GFPGAN para Análisis Forense**

Este proyecto utiliza el modelo **GFPGAN** (Generative Facial Prior Generative Adversarial Network) para mejorar la calidad de imágenes, especialmente rostros, y escalarlas al doble de su tamaño original. Está diseñado para ser utilizado en el ámbito del **cómputo forense**, donde la claridad y calidad de las imágenes son importante para el análisis e identificación.

In [ ]:
#@title #1.) Configuración inicial y creación de carpeta EVIDENCIAS
%%capture

# Desinstalar versiones actuales de torch, torchvision y torchaudio
!pip uninstall torch torchvision torchaudio -y

# Instalar versiones compatibles de torch y torchvision
!pip install torch==1.13.1 torchvision==0.14.1 --extra-index-url https://download.pytorch.org/whl/cu117

# Instalar dependencias necesarias
!pip install gfpgan
!pip install realesrgan
!pip install basicsr

# Crear carpeta EVIDENCIAS
import os
from google.colab import files
import torch
from gfpgan import GFPGANer

if not os.path.exists('EVIDENCIAS'):
    os.makedirs('EVIDENCIAS')
    print("Carpeta 'EVIDENCIAS' creada.")

# Descargar el modelo GFPGAN
print("Descargando el modelo GFPGAN...")
model_path = 'GFPGANv1.4.pth'
if not os.path.exists(model_path):
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -O GFPGANv1.4.pth

# Cargar el modelo GFPGAN
print("Cargando el modelo GFPGAN...")
restorer = GFPGANer(
    model_path=model_path,
    upscale=2,  # Escalar al doble de tamaño
    arch='clean',
    channel_multiplier=2,
    bg_upsampler=None
)
print()
print("Modelo GFPGAN cargado correctamente.")

In [ ]:
#@title #2.) Mejorar la imagen usando GFPGAN y escalarla al doble

import cv2
import matplotlib.pyplot as plt
import glob

def mejorar_imagen_gfpgan(img_path):
    """Mejorar la imagen usando GFPGAN."""
    # Leer la imagen
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"Error al cargar la imagen: {img_path}")
        return None

    # Mejorar la imagen con GFPGAN
    _, _, img_mejorada = restorer.enhance(
        img,
        has_aligned=False,
        only_center_face=False,
        paste_back=True
    )

    return img_mejorada

def mostrar_imagenes(img_original, img_mejorada):
    """Mostrar la imagen original y la mejorada."""
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title("Imagen Original")
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB))

    plt.subplot(1, 2, 2)
    plt.title("Imagen Mejorada con GFPGAN")
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img_mejorada, cv2.COLOR_BGR2RGB))
    plt.show()

# Procesar todas las imágenes en la carpeta EVIDENCIAS
input_list = glob.glob(os.path.join('EVIDENCIAS', '*'))

for input_path in input_list:
    # Leer la imagen original
    img_original = cv2.imread(input_path)
    if img_original is None:
        print(f"Error al cargar la imagen: {input_path}")
        continue

    # Mejorar la imagen con GFPGAN
    img_mejorada = mejorar_imagen_gfpgan(input_path)
    if img_mejorada is None:
        continue

    # Mostrar resultados en pantalla
    mostrar_imagenes(img_original, img_mejorada)

    # Guardar la imagen mejorada
    filename = os.path.basename(input_path)
    filename_base, _ = os.path.splitext(filename)
    output_path = os.path.join('EVIDENCIAS', f"{filename_base}_mejoradoIA.jpg")
    cv2.imwrite(output_path, img_mejorada)
    print(f"Imagen mejorada guardada en: {output_path}")

In [ ]:
#@title #3) Empaquetar imágenes en un archivo ZIP

import zipfile
from google.colab import files
import os

# Crear un archivo ZIP con las imágenes originales y mejoradas
zip_path = 'EVIDENCIAS_Mejoradas.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, dirs, files_in_folder in os.walk('EVIDENCIAS'):
        for file in files_in_folder:
            # Agregar tanto el archivo original como el mejorado
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('_mejoradoIA.jpg'):
                file_path = os.path.join(root, file)
                zipf.write(file_path, arcname=file)

print(f"Archivo ZIP creado: {zip_path}")

# Descargar el archivo ZIP
files.download(zip_path)
print(f"Archivo {zip_path} descargado.")